# OpenAlex Colombia
[OpenAlex](https://openalex.org/)  is an index of hundreds of millions of metadata of scientific articles in all areas.

We will use the  access via a API to obtain the metadata about the scientific articles of Colombia

In [8]:
import requests
import time
import json
import pandas as pd
from IPython.display import JSON

Below is an example of a JSON of the `work` entity. See the full documentation here: https://docs.openalex.org/

In [39]:
JSON({"id":"https://openalex.org/W2213304073","doi":"https://doi.org/10.1103/physrevd.93.013012",
      "title":"Fermion dark matter from SO(10) GUTs","display_name":"Fermion dark matter from SO(10) GUTs",
      "publication_year":2016,"publication_date":"2016-01-13",
      "ids":{"openalex":"https://openalex.org/W2213304073","doi":"https://doi.org/10.1103/physrevd.93.013012","mag":"2213304073"},
      "host_venue":{"id":"https://openalex.org/V4210238307","issn_l":"2470-0010","issn":["2470-0037","2470-0010","2470-0029"],
                    "display_name":"Physical review","publisher":None,"type":"repository",
                    "url":"https://doi.org/10.1103/physrevd.93.013012","is_oa":True,"version":"submittedVersion","license":None},
      "type":"journal-article","open_access":{"is_oa":True,"oa_status":"green","oa_url":"http://arxiv.org/pdf/1509.06313"},      
      "authorships":[{"author_position":"first","author":{"id":"https://openalex.org/A2998524932","display_name":"Carolina Arbeláez","orcid":None},
                      "institutions":[{"id":"https://openalex.org/I75778554","display_name":"Federico Santa María Technical University",
                                       "ror":"https://ror.org/05510vn56","country_code":"CL","type":"education"}],
                      "raw_affiliation_string":"Universidad Técnica Federico Santa María and Centro Científico-Tecnológico de Valparaíso Casilla 110-V, Valparaíso, Chile"},
                    ],
      "cited_by_count":18,
      "biblio":{"volume":"93","issue":"1","first_page":None,"last_page":None},
      "is_retracted":False,
      "is_paratext":False,                     
      "concepts":[{"id":"https://openalex.org/C121332964","wikidata":"https://www.wikidata.org/wiki/Q413","display_name":"Physics","level":0,"score":"0.9535045"},
                  {"id":"https://openalex.org/C109214941","wikidata":"https://www.wikidata.org/wiki/Q18334","display_name":"Particle physics","level":1,"score":"0.7087808"},
                  {"id":"https://openalex.org/C159249277","wikidata":"https://www.wikidata.org/wiki/Q79925","display_name":"Dark matter","level":2,"score":"0.6155488"}                  
                 ],
      
      "mesh":[],
      "alternate_host_venues":[{"id":None,"display_name":"arXiv","type":"repository","url":"http://arxiv.org/pdf/1509.06313",
                                "is_oa":True,"version":"submittedVersion","license":None}],
      "referenced_works":["https://openalex.org/W167592539","https://openalex.org/W1504118794","https://openalex.org/W1592654218","https://openalex.org/W1604802664"],
      "related_works":["https://openalex.org/W1813212307","https://openalex.org/W1951792185","https://openalex.org/W1962172028","https://openalex.org/W1977308070"],
      "ngrams_url":"https://api.openalex.org/works/W2213304073/ngrams",
      "abstract_inverted_index":{"dark":[11],"matter":[12,35],"(DM)":[13]},
      "cited_by_api_url":"https://api.openalex.org/works?filter=cites:W2213304073",
      "counts_by_year":[{"year":2022,"cited_by_count":1},{"year":2021,"cited_by_count":2},{"year":2019,"cited_by_count":1},{"year":2018,"cited_by_count":4},
                  {"year":2017,"cited_by_count":4},{"year":2016,"cited_by_count":6}],
      "updated_date":"2022-10-13T20:57:55.218364",
      "created_date":"2016-06-24"      
      })

<IPython.core.display.JSON object>

## API usage example
To use directly the obtained JSON, see next cell

In [2]:
co=[]
i=0
count=0
baseurl='https://api.openalex.org/works?filter=authorships.institutions.country_code:co&mailto=restrepo@udea.edu.co&per-page=200'
nc='*'

In [113]:
while nc:
    i+=1
    url=baseurl+f'&cursor={nc}'
    try:
        r=requests.get(url)
        cotmp=r.json().get('results')
    except:
        print('ERROR: bad url')
        break
    if i==1:
        try:
            count=r.json().get('meta').get('count')
        except:
            print('ERROR: bad count')
            break
    kk=[d.pop('abstract_inverted_index') for d in cotmp if d.get('abstract_inverted_index')]
    try:
        nc=r.json().get('meta').get('next_cursor')
    except:
        print('ERROR: bad cursor')
        break
    #progress
    print(f'{i} → {200*i}/{count}',end='\r')
    #Append to dictionary list (WARNING: HIG RAM USAGE)
    for p in cotmp:
        co.append(p)
    #=== backup =====
    f=open('openalexco.jsonl','a')
    for p in cotmp:
        json.dump(p,f)
        f.write("\n")
    f.close()
    #API limits
    time.sleep(0.1)
    #EMERGENGY EXIT
    if i*200>count:
        break

In [40]:
f=open('openalexco.json','w')
json.dump(co,f)
f.close()

In [116]:
f=open('openalexco.jsonl','a')
for p in co:
    json.dump(p,f)
    f.write("\n")
f.close()

## Use downloaded JSON

In [42]:
f=open('openalexco.json','r')
co=json.load(f)
f.close()

Medicine & Psychology Colombia

In [137]:
md=[d for d in co if len( {'Medicine','Psychology'}.intersection( 
       [ c.get('display_name') for c in d.get('concepts') if c.get('level')==0 ] 
          ) )>0 ]

In [153]:
len(md)

77352

In [216]:
f=open('openalexco_md.json','w')
json.dump(md,f)
f.close()

In [2]:
f=open('openalexco_md.json','r')
md=json.load(f)
f.close()

Medicine & Psychology Bogotá

In [3]:
insts=set()
for d in md:
    for a in d.get('authorships'):
        if a.get('institutions'):
            for i in a.get('institutions'):
                if i.get('country_code')=="CO":
                    insts.add(i.get('display_name'))

In [4]:
len(insts)

309

In [5]:
[i for i in insts if i.find('National University of Colombia')>-1]

['National University of Colombia at Medellín',
 'National University of Colombia at Manizales',
 'National University of Colombia']

In [6]:
[i for i in insts if i.find('Jav')>-1]

['Pontificia Universidad Javeriana']

In [7]:
insts_bg={'El Bosque University',
'Pontificia Universidad Javeriana',          
'National University of Colombia',
'Universidad Antonio Nariño',
'Universidad Externado de Colombia',
'Universidad de Bogotá Jorge Tadeo Lozano',
 'Universidad de Los Andes',
 'Universidad del Rosario',
 'University of La Sabana',
 'University of La Salle'}

In [8]:
mdbg=[]
for d in md:
    l=[ [i.get('display_name') for i in a.get('institutions')] for a in d.get('authorships') if a.get('institutions')]
    if len({item for sublist in l for item in sublist}.intersection(insts_bg))>0:
        mdbg.append(d)

In [9]:
len(mdbg)

23920

In [5]:
f=open('openalexco_mdbg.json','w')
json.dump(mdbg,f)
f.close()

NameError: name 'mdbg' is not defined

In [218]:
d=co[0]

Search for author

In [43]:
author='Diego Restrepo'
dr=[d for d in co if author in [ a.get('author').get('display_name') for a in d.get('authorships')]]

In [44]:
len(dr)

61

In [45]:
pd.DataFrame(dr).authorships.apply(lambda l: [a.get('author').get('id') for a in l if a.get('author').get('display_name')=='Diego Restrepo']
                                     ).str[0].unique()

array(['https://openalex.org/A4289616623',
       'https://openalex.org/A2110103959',
       'https://openalex.org/A4224864514',
       'https://openalex.org/A2306595523',
       'https://openalex.org/A4288869362',
       'https://openalex.org/A4290241673',
       'https://openalex.org/A2222954481'], dtype=object)

Search for Author ID

In [46]:
oa_id='https://openalex.org/A2306595523'
DR=[d for d in co if oa_id in [ a.get('author').get('id') for a in d.get('authorships')]]

In [47]:
len(DR)

43

In [48]:
pd.set_option('display.max_colwidth',200)
pd.DataFrame(DR)[['id','doi','title']].sort_values('title').drop_duplicates(subset=['title']).reset_index(drop=True)

,id,doi,title
0,https://openalex.org/W2194147853,https://doi.org/10.1016/j.nuclphysbps.2015.10.132,A model with a viable dark matter candidate and massive neutrinos
1,https://openalex.org/W3100627084,https://doi.org/10.1142/s0217732318500244,"Anomalous leptonic U(1) symmetry: Syndetic origin of the QCD axion, weak-scale dark matter, and radiative neutrino mass"
2,https://openalex.org/W3129415996,https://doi.org/10.1103/physrevd.103.095032,Anomaly-free Abelian gauge symmetries with Dirac scotogenic models
3,https://openalex.org/W4206129402,https://doi.org/10.1140/epjc/s10052-021-09918-8,Anomaly-free Abelian gauge symmetries with Dirac seesaws
4,https://openalex.org/W1967455469,https://doi.org/10.1103/physrevd.87.095010,Baryonic violation of R parity from anomalous U(1)H
5,https://openalex.org/W3105890208,https://doi.org/10.1016/s0370-2693(00)01067-4,Bilinear R-parity Violation and Small Neutrino Masses: a Self-consistent Framework
6,https://openalex.org/W2795094497,https://doi.org/10.1103/physrevd.97.115032,Bound-state dark matter and Dirac neutrino masses
7,https://openalex.org/W2963957043,https://doi.org/10.1016/j.physletb.2019.01.023,Bound-state dark matter with Majorana neutrinos
8,https://openalex.org/W2145042168,https://doi.org/10.1140/epjc/s10052-008-0713-4,Collider aspects of flavor physics at high Q
9,https://openalex.org/W2049677811,https://doi.org/10.1088/1126-6708/2005/03/062,Collider signals of gravitino dark matter in bilinearly broken R-parity


In [49]:
f=open('openalexco_A2306595523.json','w')
json.dump(DR,f)
f.close()

In [50]:
f=open('openalexco_A2306595523.json','r')
DR=json.load(f)
f.close()

In [51]:
[x.get('cited_by_count') for x in DR][35:]

[73, 42, 2, 22, 19, 32, 4, 4]

In [52]:
JSON(DR[35])

<IPython.core.display.JSON object>

In [53]:
def order_concepts(l):
    c=[]
    for i in range(5):
        for d in l:
            if d.get('level')==i:
                c.append(f"{d.get('level')}-{d.get('display_name')}")
    return c

In [54]:
pd.set_option('display.max_colwidth',300)

In [55]:
DRdf=pd.DataFrame(DR)

In [56]:
DRdf['c']=DRdf.concepts.apply(order_concepts)

In [57]:
DRdf.drop_duplicates(subset=['title'])[['title','c']]

,title,c
0,Anomaly-free Abelian gauge symmetries with Dirac seesaws,"[0-Physics, 1-Particle physics, 1-Theoretical physics, 2-Anomaly (physics), 2-Homogeneous space, 2-Neutrino, 2-Massless particle, 2-Physics beyond the Standard Model, 2-Symmetry (geometry), 3-Gauge symmetry, 3-Standard Model (mathematical formulation), 4-Seesaw mechanism, 4-Gauge anomaly]"
2,Phenomenology of the Zee model for Dirac neutrinos and general neutrino interactions,"[0-Physics, 1-Particle physics, 2-Neutrino, 2-Phenomenology (philosophy), 2-Physics beyond the Standard Model, 3-Neutrino oscillation, 3-Lepton, 3-Standard Model (mathematical formulation), 3-Dirac (video compression format), 4-Sterile neutrino, 4-Solar neutrino]"
3,Anomaly-free Abelian gauge symmetries with Dirac scotogenic models,"[0-Physics, 1-Particle physics, 1-Theoretical physics, 1-Mathematical physics, 2-Anomaly (physics), 2-Massless particle, 2-Neutrino, 3-Dirac fermion, 3-Gauge symmetry, 3-Standard Model (mathematical formulation), 4-Gauge anomaly]"
4,Phenomenological consistency of the singlet-triplet scotogenic model,"[0-Physics, 1-Particle physics, 1-Nuclear physics, 2-Dark matter, 2-Neutrino, 2-Annihilation, 2-Parameter space, 2-Fermi Gamma-ray Space Telescope, 2-Nucleon, 2-Physics beyond the Standard Model, 3-Lepton, 3-Standard Model (mathematical formulation), 3-Singlet state]"
5,Dirac neutrino mass generation from a Majorana messenger,"[0-Physics, 1-Particle physics, 2-Neutrino, 3-MAJORANA, 3-Dirac (video compression format)]"
6,Métricas de Vinculación Universidad-Entorno: Universidad de Antioquia (Metrics for University Linkage with the Environment: Universidad De Antioquia),"[0-Geography, 1-Humanities, 1-Cartography]"
7,Singlet-doublet Dirac dark matter and neutrino masses,"[0-Physics, 1-Particle physics, 2-Neutrino, 2-Dark matter, 2-Fermion, 3-Dirac (video compression format), 3-Lepton, 3-Neutrino oscillation]"
8,"Two-component dark matter and a massless neutrino in a new <mml:math xmlns:mml=""http://www.w3.org/1998/Math/MathML"" display=""inline""><mml:mi>B</mml:mi><mml:mo>−</mml:mo><mml:mi>L</mml:mi></mml:math> model","[0-Physics, 1-Particle physics, 1-Nuclear physics, 2-Massless particle, 2-Neutrino, 2-Dark matter, 2-Component (thermodynamics)]"
9,Bound-state dark matter with Majorana neutrinos,"[0-Physics, 1-Particle physics, 2-Neutrino, 2-Dark matter, 2-Fermion, 2-Bound state, 3-MAJORANA, 3-Neutrino oscillation, 3-Lepton, 4-Sterile neutrino]"
10,"Probing a simplified W ′ model of R ( D ( * ) ) anomalies using b tags, τ leptons, and missing energy","[0-Physics, 1-Particle physics, 1-Nuclear physics, 2-Quark, 2-Large Hadron Collider, 2-Charm (quantum number), 2-Sensitivity (control systems), 2-Meson, 3-Lepton, 3-Charm quark, 3-B meson, 4-Missing energy]"
